# Fish Net Classification

In [3]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # specify which GPU(s) to be used
import pandas as pd
import numpy as np

import torch
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

import glob
from PIL import Image
import albumentations
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
sns.set_style("white")

from train import main, SalmonDataset

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

data_path = "/media/slowSSD/Carribean_Challenge/data/fish"

## Resizing

In [3]:
images = glob.glob("{}/nets/*.png".format(data_path))
labels = [1]*len(images)
how_many_nets = len(labels)
print("{:3} images with nets present".format(how_many_nets))

images += glob.glob("{}/no_nets/*.png".format(data_path))
how_many_without_nets = len(images)-how_many_nets
labels += [0]*how_many_without_nets
print("{:3} images without nets present".format(how_many_without_nets))

In [17]:
img = Image.open(images[-1])
img.size

In [13]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing
cpus = multiprocessing.cpu_count()
cpus = min(48, cpus)
print("{} cpus".format(cpus))

32 cpus


In [14]:
# sz = (450, 432)
sz = (900, 864)

In [15]:
replace_string = 'nets'
replace_string_with = "nets_{}x{}".format(sz[0], sz[1])

In [16]:
shutil.rmtree("{}/nets_{}x{}".format(data_path, sz[0], sz[1]))
shutil.rmtree("{}/no_nets_{}x{}".format(data_path, sz[0], sz[1]))

os.makedirs("{}/nets_{}x{}".format(data_path, sz[0], sz[1]), exist_ok=True)
os.makedirs("{}/no_nets_{}x{}".format(data_path, sz[0], sz[1]), exist_ok=True)

In [17]:
def resize_img(im, fn):
    im = im.resize(sz, resample=Image.BICUBIC)
    im.save(fn.replace(replace_string, replace_string_with))

def resizer_img(fn):
    im = Image.open(fn).convert("RGB")
    print("{:50} -> {:50}: {}".format(fn, fn.replace(replace_string, replace_string_with), im.size))
    resize_img(im, fn)

def resize_imgs(img_paths):
    with ProcessPoolExecutor(cpus) as e: e.map(resizer_img, img_paths)

In [4]:
resize_imgs(images)

## Training - much more data, 900x864

In [19]:
# sz = (450, 432)
sz = (900, 864)
random_crop = 448
val_percentage = 0.15

# IMAGES WITH NETS
images = glob.glob("{}/nets_{}x{}/*.png".format(data_path, sz[0], sz[1]))
labels = [1]*len(images)
how_many_nets = len(labels)
print("{:3} images with nets present".format(how_many_nets))

# VAL/TRAIN SPLIT
np.random.seed(1338)
np.random.shuffle(images)
np.random.seed(1338)
np.random.shuffle(labels)

train_images = images[:int((1-val_percentage)*len(images))]
train_labels = labels[:int((1-val_percentage)*len(labels))]
val_images = images[int((1-val_percentage)*len(images)):]
val_labels = labels[int((1-val_percentage)*len(labels)):]

# IMAGES WITHOUT NETS
images = glob.glob("{}/no_nets_{}x{}/*.png".format(data_path, sz[0], sz[1]))
how_many_without_nets = len(images)
labels = [0]*how_many_without_nets
print("{:3} images without nets present".format(how_many_without_nets))

# VAL/TRAIN SPLIT
np.random.seed(1339)
np.random.shuffle(images)
np.random.seed(1339)
np.random.shuffle(labels)

train_images += images[:int((1-val_percentage)*len(images))]
train_labels += labels[:int((1-val_percentage)*len(labels))]
val_images += images[int((1-val_percentage)*len(images)):]
val_labels += labels[int((1-val_percentage)*len(labels)):]

print("#"*60)
print("{:3} Train images --> {:2} nets = {:.1f}%".format(len(train_images), sum(train_labels), 100*sum(train_labels)/len(train_images)))
print("{:3} Valid images --> {:2} nets = {:.1f}%".format(len(val_images), sum(val_labels), 100*sum(val_labels)/len(val_images)))

111 images with nets present
7531 images without nets present
############################################################
6495 Train images --> 94 nets = 1.4%
1147 Valid images --> 17 nets = 1.5%


In [20]:
train_dataset_raw = SalmonDataset(images=train_images, targets=train_labels) 

train_transforms = []
train_transforms.append(albumentations.RandomCrop(random_crop, random_crop))
train_dataset = SalmonDataset(images=train_images, targets=train_labels, 
                              transforms=albumentations.Compose(train_transforms)) 

val_transforms = []
val_dataset = SalmonDataset(images=val_images, targets=val_labels, transforms=albumentations.Compose(val_transforms)) 

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=10, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=10, pin_memory=True)

__Visualize__

In [21]:
for j in range(10):
    rand_int = np.random.randint(len(train_dataset))
    f, axarr = plt.subplots(1, 4, figsize=(30, 12))
    sample = train_dataset_raw[rand_int]
    axarr[0].imshow(np.transpose((sample["image"].numpy()*255).astype(np.uint8), (1,2,0)))
    if sample["target"]==1: axarr[0].set_title(classes[0])
    else: axarr[0].set_title("No net")
    print(train_images[rand_int])
    axarr[1].imshow(np.transpose((train_dataset[rand_int]["image"].numpy()*255).astype(np.uint8), (1,2,0)))
    axarr[2].imshow(np.transpose((train_dataset[rand_int]["image"].numpy()*255).astype(np.uint8), (1,2,0)))
    axarr[3].imshow(np.transpose((train_dataset[rand_int]["image"].numpy()*255).astype(np.uint8), (1,2,0)))
    plt.show()

In [22]:
epochs = 1000
print_every_x_batches = 1000
lr = 0.001
arch = "resnet18"

In [23]:
patience = 3
model_name = "{}_{}nets_{}wo_val_v1_resize_{}x{}_rc_{}_pat{}".format(arch, sum(train_labels), len(train_labels)-sum(train_labels), sz[0], sz[1], random_crop, patience)
print(model_name)
main(arch=arch, model_name=model_name, train_loader=train_loader, val_loader=val_loader, epochs=epochs, lr=lr, patience=patience, print_every_x_batches=print_every_x_batches)
torch.cuda.empty_cache()

resnet18_94nets_6401wo_val_v1_resize_900x864_rc_448_pat3
Epoch: [0]	Time  0.173	Data  0.009	Loss 8.2861e-02	Acc@1 0.9800
Val: Loss 0.0641 Acc@1 0.9852
Epoch: [1]	Time  0.157	Data  0.009	Loss 5.7166e-02	Acc@1 0.9851
Val: Loss 0.0727 Acc@1 0.9852
Epoch: [2]	Time  0.159	Data  0.010	Loss 5.7423e-02	Acc@1 0.9848
Val: Loss 0.0581 Acc@1 0.9852
Epoch: [3]	Time  0.158	Data  0.009	Loss 5.0033e-02	Acc@1 0.9848
Val: Loss 0.0580 Acc@1 0.9843
Epoch: [4]	Time  0.158	Data  0.009	Loss 4.8404e-02	Acc@1 0.9846
Val: Loss 0.0706 Acc@1 0.9808
Epoch: [5]	Time  0.159	Data  0.010	Loss 5.2654e-02	Acc@1 0.9846
Val: Loss 0.1485 Acc@1 0.9582
Epoch: [6]	Time  0.159	Data  0.009	Loss 4.2602e-02	Acc@1 0.9860
Val: Loss 0.0606 Acc@1 0.9852
Epoch: [7]	Time  0.159	Data  0.009	Loss 4.3537e-02	Acc@1 0.9865
Val: Loss 0.0475 Acc@1 0.9869
Epoch: [8]	Time  0.159	Data  0.010	Loss 3.0870e-02	Acc@1 0.9909
Val: Loss 0.0547 Acc@1 0.9843
Epoch: [9]	Time  0.159	Data  0.010	Loss 2.9969e-02	Acc@1 0.9912
Val: Loss 0.0620 Acc@1 0.9852
Epo